### Set and Card Counts
This script scrapes bulbapedia to generate a list of pokemon card sets and their respective card counts. It currently scrapes only the English sets, as the Japanese cards don't have numbers or set numbers. This script should be run once daily to check for new sets, and add them to the db.

In [1]:
# imports
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


sets_url = 'https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_Trading_Card_Game_expansions'
r_sets = requests.get(sets_url, verify=False)
soup_sets = BeautifulSoup(r_sets.text, "html.parser")

bulbapedia_base_url = 'https://bulbapedia.bulbagarden.net'  # base bulbapedia url

# list of all sets in <a href="..." title="<set name> (TCG)">set name</a> form
set_link_list = soup_sets.select('tbody > tr > td > a[href*="_(TCG)"]')
href_list_full = [el['href'] for el in set_link_list]  # get '/wiki/<set>_(TCG)' text from url
# generate url from href and base url
url_list_full = [bulbapedia_base_url + el for el in href_list_full]

# break the url list off at "Wizards Black Star Promos"
wizards_re_url = re.compile('.*/wiki/Wizards.*')  # not flexible if a set with "Wizards" in name drops
wizards_promo_href = [string for string in url_list_full if re.match(wizards_re_url, string)]
wizards_promo_href_ix = url_list_full.index(wizards_promo_href[0])
url_list = url_list_full[:wizards_promo_href_ix]  # grab everything in url_list_full up to Wizards promo

# create a list of set names
set_name_list = [el.split('/')[-1].replace('_', ' ').replace('%26', '&')[:-6] for el in url_list]

# generate list of requests from url_list
r_list = [requests.get(url, verify=False) for url in url_list]
# generate soup list from r_list
r_soup_list = [BeautifulSoup(r.text, 'html.parser') for r in r_list]

'''
find the table at the link for each set with #/### and card name. There a few different cases here, 
so we start with a tmp table, then filter it down to make the real table. The different cases are:

1) Set with English and Japanese versions
Two or more tables with card no. and card name under "Set list" header on bulbapedia. The key 
here is to look for elements with style="text-align:left"
ex. Base Set

2) Set with NO Japanese version
One table with card no. and card name under "Set list" header on bulbapedia. The key here is to
look for elements with style="float:left". There are also instances where more than one column exists,
like with the "Diamond & Pearl" set. In this case, the 'style="float:left"' logic fails. Instead,
we want the 'style="text-align:left"' logic. This is because the former logic only picks up the
header(s) of the table(s)
ex. Base Set 2

3) Set with NO English version
We want to remove these completely. These are captured by the "len(el) == 1" logic, since the scraper
only pulls the set name from the table header by selecting "style='text-align:left'"
ex. Pokémon Card★VS

4) Set with English and Japanese versions, where Japanese version has a link
This is a set that will get picked up by our scraper, so we want to make sure to remove it. This is
done later in the script when we clean up the elements returned in our soup
ex. Moonlit Pursuit/Dawn Dash
'''

card_tbl_list_tmp = [el.select("[style*='text-align:left']") for el in r_soup_list]

'''
list of char counts for elements grabbed by the select from r_soup_list above. It looks something
like this: [[10000, 80], [70, 85], [81], [20000, 70, 25000, 80, 82], ...]. We will remove the
[#] elements, as they're solely Japanese sets. We will change the "select" statement for the
[<200, <200, ...] elements, since they picked up the table headers, not the actual content (case 2).
Otherwise, we keep the elements, as they contain the data we're after
'''

char_ct_list = []
for card_tbl in card_tbl_list_tmp:
    tmp_list = []
    for el in card_tbl:
        tmp_list.append(len(str(el)))
    char_ct_list.append(tmp_list)

# list elements of the form [(<set name>, request element(s), [#, #, ...]), ("), ...]
name_tbl_ct_tuple = list(map(lambda x,y,z:(x,y,z), set_name_list, card_tbl_list_tmp, char_ct_list))
    
# create the final card table list for each set
name_list = []
tbl_list = []
for ix,el in enumerate(name_tbl_ct_tuple):
    if len(el[2]) == 1:  # remove "Pokémon Card★VS" and "Pokémon Card★web" Japanese sets
        pass
    # handle sets without Japanese versions, as well as instances where the scraper fails to
    # pick up the content of the "Set list" table and instead grabs the table header(s)
    elif all([num < 200 for num in el[2]]):
        tbl_content_tmp = r_soup_list[ix].select("[style*='float:left']")
        name_list.append(el[0])
        tbl_list.append(tbl_content_tmp)
    else:  # keep what's already in card_tbl_list_tmp
        name_list.append(el[0])
        tbl_list.append(el[1])      
                
# turn the soup into text for easy parsing
raw_txt_list = []
for col in tbl_list:
    tmp_list = []
    for el in col:
        tmp_list.append(el.text)
    raw_txt_list.append(tmp_list)

# grab all the elements with "/" in them - we are looking for #/### to get the # of cards in the set
sets_txt_list = []
for group in raw_txt_list:
    tmp_list = []
    for txt in group:
        if '/' in txt: tmp_list.append(txt)
    sets_txt_list.append(tmp_list)
    
# remove '\n' and whitespace from sets_txt_list
sets_txt_clean_list = []
name_remove_ixs = []
for ix,list_ in enumerate(sets_txt_list):
    tmp_list = []
    for el in list_:
        tmp_list.append(el.replace('\n', ' '))
    if len(tmp_list) > 0:  # removes Moonlit Pursuit/Dawn Dash set(s)
        sets_txt_clean_list.append(tmp_list)
    else:
        name_remove_ixs.append(ix)
        
# remove the set(s) from name_list that don't have values in their cleaned text list
for ix,ix_to_pop in enumerate(name_remove_ixs):
    # if there is more than one element to pop, this makes sure we pop the right one as name_list 
    # gets shorter
    name_list.pop(ix_to_pop - ix)

# create list of strings in sets_txt_clean_list for ease of parsing to search for ###/###
sets_txt_clean_strs_list = [el[0].split() for el in sets_txt_clean_list]

set_ct_re = re.compile('\d+\/\d+')  # regex to match ###/### format
set_ct_list = []
# pull
for el in sets_txt_clean_strs_list:
    tmp_list = []
    for string in el:
        if re.match(set_ct_re, string):
            tmp_list.append(string)
    set_ct_list.append(int(tmp_list[0].split('/')[1]))  # pull the ### right of the slash

# final list that has the set name along with how many cards are in it!!
name_ct_list = list(map(lambda x,y:(x,y), name_list, set_ct_list))

# create a pd df
set_df = pd.DataFrame({'Set Name': name_list, 'Card Count': set_ct_list})

# save it to an excel
set_df.to_excel('Set_and_Card_Ct.xlsx')